# 背景介绍

打算在重庆买房子，住房信息在政府官网都能查到

重庆的新房信息可以在网址http://www.cq315house.com/ 上查到(登记备案了的)

于是打算爬下数据，便于快速筛选

# 基本设置

In [1]:
import requests
import re,json
import pandas as pd
import demjson
# http://deron.meranda.us/python/demjson/
# http://blog.csdn.net/blueheart20/article/details/69704518
import time

In [2]:
# 这里定义一个函数，将网址转化成html格式
# 区分get,post模式

import requests

def url2html(url,type = 'get',encoding='utf-8'):
    # 默认设置模式为get，编码格式为 utf-8
    try:
        if type == 'get':
            r = requests.get(url,timeout=120)
        elif type == 'post':
            r = requests.post(url,timeout=120)
    except requests.exceptions.ConnectTimeout:
        print('Wrong URL! Please Check')
    else:
        if r.status_code == 200:
            r.encoding = encoding
            content = r.text
        return content

# 参数解析

参数解析
网址检索参数很好地对应了搜索框输入的内容
http://www.cq315house.com/315web/webservice/GetMyData999.ashx?projectname=\
    &site=\
    &kfs=\
    &projectaddr=\
    &pagesize=10\
    &pageindex=2\
    &roomtype=\
    &buildarea=3

buildarea 建筑面积
- 1 0-70
- 2 70-90
- 3 90-120
- 4 120-140
- 5 140-180
- 6 180-210
- 7 210以上

site 区县
- 巴南
- 北碚
- 大渡口
- 江北
- 九龙坡
- 南岸
- 沙坪坝
- 渝北
- 渝中
- 两江新

roomtype 房屋用途
- 住宅
- 办公
- 商铺
- 其他

In [3]:
# 渝北，住宅，90-120
site = u'渝北'
roomtype = u'住宅'
kfs = u''
projectaddr = u''
page_idx = str(1)
url = 'http://www.cq315house.com/315web/webservice/GetMyData999.ashx?projectname=&site='\
    +site\
    +'&kfs='+kfs\
    +'&projectaddr='+projectaddr\
    +'&pagesize=50'\
    +'&pageindex='+page_idx\
    +'&roomtype='+roomtype\
    +'&buildarea=3'

In [4]:
url

'http://www.cq315house.com/315web/webservice/GetMyData999.ashx?projectname=&site=渝北&kfs=&projectaddr=&pagesize=50&pageindex=1&roomtype=住宅&buildarea=3'

# 页面循环

In [5]:
# 房屋信息筛选
# 区域不限
site = u''
# 房屋用途，住宅
roomtype = u'住宅'
# 面积范围
# 2表示70-90,3表示90-120
buildarea = '3' 

# 是否继续
mark = 1
# 页面迭代
page_idx = 1
# 信息存储
house_list = list()

while mark:
    print('Scanning page %d' %page_idx)
    url = 'http://www.cq315house.com/315web/webservice/GetMyData999.ashx?projectname=&site='\
        +site\
        +'&kfs='\
        +'&projectaddr='\
        +'&pagesize=50'\
        +'&pageindex='+str(page_idx)\
        +'&roomtype='+roomtype\
        +'&buildarea='+buildarea
    html = url2html(url = url,type = 'post')
    if len(html)>5:
        # 去除特殊字符
        # 不然可能报错 Line terminator characters must be escaped inside string literals'
        # https://stackoverflow.com/questions/19176024/how-to-escape-special-characters-in-building-a-json-string
        html = re.sub(r'[\r|\n|\t]','', html)  
        house_tmp = demjson.decode(html)
        house_list.extend(house_tmp)
        page_idx += 1
    else:
        mark = 0
print('%d info found'%len(house_list))

Scanning page 1
Scanning page 2
Scanning page 3
Scanning page 4
Scanning page 5
Scanning page 6
Scanning page 7
Scanning page 8
Scanning page 9
Scanning page 10
Scanning page 11
Scanning page 12
Scanning page 13
Scanning page 14
Scanning page 15
Scanning page 16
Scanning page 17
Scanning page 18
Scanning page 19
Scanning page 20
928 info found


In [6]:
house_df = pd.DataFrame(house_list)
print(house_df.shape)

(928, 17)


In [7]:
# 保存数据
file_data = 'CQ_house_info_90-120_20180119.xlsx'
house_df.to_excel(file_data,index=False)

In [8]:
# 载入先前已存数据
house_df = pd.read_excel(file_data)

In [9]:
# 查询特定楼盘的信息
# http://www.cq315house.com/315web/webservice/GetBuildingInfo.ashx?buildingId=11113985

buildingId = str(11113985)
url = 'http://www.cq315house.com/315web/webservice/GetBuildingInfo.ashx?buildingId=' + buildingId
html = url2html(url,type = 'post')
html = re.sub(r'[\n|\r|\t]','',html)
buildingInfo_tmp = demjson.decode(html)

In [11]:
buildingInfo_tmp

{'enterpriseName': '重庆亿臻房地产开发有限公司',
 'enterpriseOrgCode': '',
 'enterpriseDelegate': '唐云龙',
 'projectName': '琅樾江山H06-2-2地块',
 'location': '大渡口区铂桥路56号',
 'presaleCert': '渝国土房管（2018）预字第（1767）号'}